In [16]:
from glob import glob
import pandas as pd
import pickle

In [3]:

def go():
    for filename  in glob('../results/results_w_top_features/*top_features*.txt'):
        _, feature_set, _, state = filename.split('/')[-1].replace('[top_features].txt', '').split('_')
        with open(filename) as f:
            for feature_rank, l in enumerate(f):
                feature_name = l.strip()
                yield feature_set, state, feature_name, feature_rank
df = pd.DataFrame(list(go()), columns=['feature_set', 'land', 'feature', 'feature_rank'])
df.loc[df.feature_set == 'kw', 'feature_set'] = 'yake'

In [4]:
def get_simple_feature(babel_feature):
    try:
        f = json.loads(babel_feature)
    except:
        return None
    if f['dbpedia']:
        return 'babel/dbpedia: ' + urllib.parse.unquote(f['dbpedia'][0].split('/')[-1])
    else:
        return 'babel/text: ' + urllib.parse.unquote(f['text'][0])
    
df['simple_feature'] = df.feature_set + ': ' + df.feature    
df.loc[df.feature_set == 'babelkw', 'simple_feature'] = df.feature.apply(get_simple_feature)


In [5]:
df.feature_set.unique()

array(['wikikw', 'babelkw', 'yake'], dtype=object)

In [6]:
df[df.feature_set == 'yake'].groupby('feature').feature_rank.median().sort_values(ascending=True)[:50]

feature
Grundwissen                   0.5
Blick                         1.0
Energie                       4.0
Körper                        4.5
Licht                         7.0
Abb                           7.5
Wichtigste                    7.5
Kraft                         9.0
Bewegung                     10.5
Natur                        19.0
Mathematiker und Physiker    21.0
Wärmetransport               21.0
Teilchen                     21.5
Animation in Abb             24.0
Induktion                    24.0
Strom                        24.5
Erde                         25.5
Sonne                        26.5
Volumen                      26.5
Mond                         27.0
Kristall                     27.0
Linse                        27.5
Aufgabe                      28.0
unserem Gehirn               28.0
Temperatur                   28.5
Volumenänderung              29.0
Linsen                       34.0
Druck                        35.0
Aufbau                       36.5
Gegens

In [10]:
df[df.feature_set == 'wikikw'].groupby('feature').feature_rank.median().sort_values(ascending=True)[:50]

feature
Elektron                         5.5
Visuelle Wahrnehmung             7.5
Elektrischer Strom               8.5
Magnetismus                      9.0
Energie                         11.0
Wärmeleitung                    15.5
Elektromagnetische Induktion    15.5
Licht                           16.0
Mond                            16.5
Atom                            18.5
Beschleunigung                  19.0
Luftdruck                       20.5
Himmelspol                      21.0
Tragfläche                      21.0
Hornhaut                        21.0
Elementarladung                 23.5
Glühlampe                       24.5
Dichte                          26.5
Linse (Optik)                   27.5
Gewichtskraft                   28.0
Kraft                           29.5
Elektrische Ladung              30.0
Schall                          33.0
Anekdote                        33.0
Spektralfarbe                   33.0
Schwingung                      34.5
Statischer Auftrieb           

In [7]:
babel_features = df[df.feature_set == 'babelkw'].groupby('feature').feature_rank.median().sort_values(ascending=True)[:50]

In [8]:
import json

In [12]:
def get_text_rep(f):
    f = json.loads(f)
    if len(f['dbpedia']) >= 1:
        return f['dbpedia'][0]
    else:
        return f['text'][0]
        
top_features = df[df.feature_set == 'babelkw'].groupby('feature').feature_rank.std().sort_values(ascending=True).index
top_resources = [e for e in [get_text_rep(f) for f in top_features] if 'resource' in e]

In [14]:
for r in top_resources:
    print(r)

http://dbpedia.org/resource/Thermodynamic_free_energy
http://dbpedia.org/resource/Force
http://dbpedia.org/resource/Diagram
http://dbpedia.org/resource/Button
http://dbpedia.org/resource/Physical_body
http://dbpedia.org/resource/Light
http://dbpedia.org/resource/Fundamental_science
http://dbpedia.org/resource/Equality_(mathematics)
http://dbpedia.org/resource/Animation
http://dbpedia.org/resource/Inquiry
http://dbpedia.org/resource/Alpha
http://dbpedia.org/resource/Ohm's_law
http://dbpedia.org/resource/Armature_(electrical_engineering)
http://dbpedia.org/resource/Research
http://dbpedia.org/resource/Motion_(physics)
http://dbpedia.org/resource/Time
http://dbpedia.org/resource/Particle
http://dbpedia.org/resource/Experiment
http://dbpedia.org/resource/Voltage
http://dbpedia.org/resource/Tycho_Brahe
http://dbpedia.org/resource/Oscillation
http://dbpedia.org/resource/Cloud_chamber
http://dbpedia.org/resource/Water
http://dbpedia.org/resource/Thermodynamic_equilibrium
http://dbpedia.org/re

In [17]:
with open('../data/cache/categories_depth_5.pkl', 'rb') as f:
    categories_for_resouce = pickle.load(f)

In [18]:
def is_accepted_topics(c):
    return ('http://dbpedia.org/resource/Category:Subfields_of_physics' in c) or \
    ('http://dbpedia.org/resource/Category:Physics' in c) or \
    ('http://dbpedia.org/resource/Category:Concepts_in_physics' in c) or \
    ('http://dbpedia.org/resource/Category:Physical_sciences' in c) or \
    ('http://dbpedia.org/resource/Category:Electromagnetism' in c) or \
    ('http://dbpedia.org/resource/Category:Electrical_engineering' in c) or \
    ('http://dbpedia.org/resource/Category:Physical_quantities' in c) or \
    ('http://dbpedia.org/resource/Category:Universe' in c) or \
    ('http://dbpedia.org/resource/Category:Metrology' in c) or \
    ('http://dbpedia.org/resource/Category:Classical_mechanics' in c) or \
    ('http://dbpedia.org/resource/Category:Engineering_disciplines' in c) or \
    ('http://dbpedia.org/resource/Category:Applied_and_interdisciplinary_physics' in c) or \
    ('http://dbpedia.org/resource/Category:Applied_sciences' in c) or \
    ('http://dbpedia.org/resource/Category:Engineering_disciplines' in c)

In [19]:
relevant_category = [any([is_accepted_topics(c) for c in categories_for_resouce[r]]) 
                     for r in top_resources]

In [20]:
len(relevant_category)

680

In [27]:

w_rel_cat = pd.DataFrame({'is_relevant_catgory': relevant_category, 'feature': top_resources})
w_rel_cat[w_rel_cat.is_relevant_catgory].shape[0] / w_rel_cat.shape[0]

0.9029411764705882

In [37]:
w_rel_cat[~w_rel_cat.is_relevant_catgory].feature.iloc[:30]

6        http://dbpedia.org/resource/Fundamental_science
7      http://dbpedia.org/resource/Equality_(mathemat...
10                     http://dbpedia.org/resource/Alpha
13                  http://dbpedia.org/resource/Research
43                 http://dbpedia.org/resource/Rectangle
52                http://dbpedia.org/resource/Role_model
85                       http://dbpedia.org/resource/Man
94             http://dbpedia.org/resource/Total_(group)
103              http://dbpedia.org/resource/Calculation
104                      http://dbpedia.org/resource/Lag
120                  http://dbpedia.org/resource/Looking
125                 http://dbpedia.org/resource/Triangle
127               http://dbpedia.org/resource/Phenomenon
131                   http://dbpedia.org/resource/System
139             http://dbpedia.org/resource/Line_segment
143      http://dbpedia.org/resource/Grading_(education)
153     http://dbpedia.org/resource/Sides_of_an_equation
156                   http://db